In [101]:
import os
import json
import pandas as pd


In [95]:
cd "C:\Users\chidi\OneDrive\Desktop\sideProjects\PokeSim"

C:\Users\chidi\OneDrive\Desktop\sideProjects\PokeSim


In [103]:
# First, most basic feature: pokémon difference between p1 and p2
# => model
# + sorted list of HP remaining
# => Model
# + base stat totals
# => Model
# Map moves to types (think of a smarter feature)
# => Model
# Additional features: status? active mon speed? Boosts?

import os
import json
import pandas as pd


def parsePlayer(pokemonList, side):
    player = pokemonList[side]
    playerList = player["pokemon"]
    pokeState = {}
    for idx, pokemon in enumerate(playerList):
        pokeState = _parse_nameStatusTypeHp(side, pokeState, idx, pokemon)
        pokeState = _parse_moves(side, pokeState, idx, pokemon)
        pokeState = _parse_stats(side, pokeState, idx, pokemon)
        pokeState = _parse_types(side, pokeState, idx, pokemon)

    pokeState[f"p{side+1}pkmn{idx+1}pkmLeft"] = player["pokemonLeft"]
    return pokeState


def _parse_nameStatusTypeHp(side, pokeState, idx, pokemon):
    pokeState[f"p{side+1}pkmn{idx+1}name"] = pokemon["speciesState"]["id"]
    pokeState[f"p{side+1}pkmn{idx+1}status"] = pokemon["status"]
    pokeState[f"p{side+1}pkmn{idx+1}type{1}"] = pokemon["types"][0]
    pokeState[f"p{side+1}pkmn{idx+1}RemainingHp"] = pokemon["hp"]
    return pokeState


def _parse_stats(side, pokeState, idx, pokemon):
    for stat, value in pokemon["modifiedStats"].items():
        pokeState[f"p{side+1}pkmn{idx+1}{stat}"] = value
    return pokeState


def _parse_moves(side, pokeState, idx, pokemon):
    for moveidx, move in enumerate(pokemon["set"]["moves"]):
        pokeState[f"p{side+1}pkmn{idx+1}move{moveidx+1}"] = move
    if len(pokemon["set"]["moves"]) < 4:
        movesToFill = 4 - len(pokemon["set"]["moves"])
        for i in range(movesToFill):
            moveidx = movesToFill - i
            pokeState[f"p{side+1}pkmn{idx+1}move{moveidx+1}"] = "NoMove"
    return pokeState


def _parse_types(side, pokeState, idx, pokemon):
    if len(pokemon["types"][0]) == 2:
        pokeState[f"p{side+1}pkmn{idx+1}type{2}"] = pokemon["types"][1]
    else:
        pokeState[f"p{side+1}pkmn{idx+1}type{2}"] = pokemon["types"][0]
    return pokeState

def parseMatchTurns(folderName="data"):
    parsedMatchTurns = []
    dataPath = os.path.join(os.getcwd(), folderName)
    matchTurns = os.listdir(dataPath)

    for matchTurn in matchTurns:
        batlePath = os.path.join(dataPath, matchTurn)
        with open(batlePath) as file:
            sides = json.load(file)["sides"]
        p1Data = parsePlayer(sides, side=0)
        p2Data = parsePlayer(sides, side=1)
        turnData = {**p1Data, **p2Data}
        turnData["match_turn"] = matchTurn

        parsedMatchTurns.append(turnData)

    return pd.DataFrame(parsedMatchTurns)




,p1pkmn1name,p1pkmn1status,p1pkmn1type1,p1pkmn1RemainingHp,p1pkmn1move1,p1pkmn1move2,p1pkmn1move3,p1pkmn1move4,p1pkmn1atk,p1pkmn1def,...,p2pkmn6move3,p2pkmn6move4,p2pkmn6atk,p2pkmn6def,p2pkmn6spa,p2pkmn6spd,p2pkmn6spe,p2pkmn6type2,p2pkmn6pkmLeft,match_turn
0,paras,,Bug,241,stunspore,spore,megadrain,bodyslam,210,183,...,thunderwave,bodyslam,184,189,207,207,169,Normal,6,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
1,graveler,,Rock,252,earthquake,bodyslam,explosion,rockslide,231,263,...,thunderwave,bodyslam,184,189,207,207,169,Normal,6,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
2,paras,,Bug,145,stunspore,spore,megadrain,bodyslam,210,183,...,sleeppowder,stunspore,138,199,169,169,138,Grass,4,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
3,mewtwo,,Psychic,72,psychic,amnesia,thunderbolt,blizzard,199,174,...,sleeppowder,stunspore,138,199,169,169,138,Grass,4,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
4,chansey,,Normal,98,sing,softboiled,thunderwave,icebeam,75,75,...,thunderwave,bodyslam,184,189,207,207,169,Normal,2,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11095,charizard,,Fire,101,hyperbeam,swordsdance,fireblast,bodyslam,205,196,...,surf,seismictoss,171,201,174,174,40,Water,1,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11096,moltres,,Fire,297,reflect,fireblast,hyperbeam,agility,230,215,...,surf,seismictoss,171,201,174,174,40,Water,1,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11097,moltres,,Fire,297,reflect,fireblast,hyperbeam,agility,230,215,...,surf,seismictoss,171,201,174,174,40,Water,0,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11098,charizard,,Fire,278,hyperbeam,swordsdance,fireblast,bodyslam,205,196,...,agility,bodyslam,176,161,165,165,169,Normal,6,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."


In [99]:
cleanedData = parseMatchTurns("data")


,p1pkmn1name,p1pkmn1status,p1pkmn1type1,p1pkmn1type2,p1pkmn1RemainingHp,p1pkmn1move1,p1pkmn1move2,p1pkmn1move3,p1pkmn1move4,p1pkmn1atk,...,p2pkmn6move2,p2pkmn6move3,p2pkmn6move4,p2pkmn6atk,p2pkmn6def,p2pkmn6spa,p2pkmn6spd,p2pkmn6spe,p2pkmn6pkmLeft,match_turn
0,paras,,Bug,Bug,241,stunspore,spore,megadrain,bodyslam,210,...,blizzard,thunderwave,bodyslam,184,189,207,207,169,6,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
1,graveler,,Rock,Rock,252,earthquake,bodyslam,explosion,rockslide,231,...,blizzard,thunderwave,bodyslam,184,189,207,207,169,6,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
2,paras,,Bug,Bug,145,stunspore,spore,megadrain,bodyslam,210,...,psychic,sleeppowder,stunspore,138,199,169,169,138,4,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
3,mewtwo,,Psychic,Psychic,72,psychic,amnesia,thunderbolt,blizzard,199,...,psychic,sleeppowder,stunspore,138,199,169,169,138,4,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
4,chansey,,Normal,Normal,98,sing,softboiled,thunderwave,icebeam,75,...,blizzard,thunderwave,bodyslam,184,189,207,207,169,2,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11095,charizard,,Fire,Fire,101,hyperbeam,swordsdance,fireblast,bodyslam,205,...,blizzard,surf,seismictoss,171,201,174,174,40,1,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11096,moltres,,Fire,Fire,297,reflect,fireblast,hyperbeam,agility,230,...,blizzard,surf,seismictoss,171,201,174,174,40,1,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11097,moltres,,Fire,Fire,297,reflect,fireblast,hyperbeam,agility,230,...,blizzard,surf,seismictoss,171,201,174,174,40,0,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11098,charizard,,Fire,Fire,278,hyperbeam,swordsdance,fireblast,bodyslam,205,...,swordsdance,agility,bodyslam,176,161,165,165,169,6,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."


In [100]:
cleanedData

,p1pkmn1name,p1pkmn1status,p1pkmn1type1,p1pkmn1type2,p1pkmn1RemainingHp,p1pkmn1move1,p1pkmn1move2,p1pkmn1move3,p1pkmn1move4,p1pkmn1atk,...,p2pkmn6move2,p2pkmn6move3,p2pkmn6move4,p2pkmn6atk,p2pkmn6def,p2pkmn6spa,p2pkmn6spd,p2pkmn6spe,p2pkmn6pkmLeft,match_turn
0,paras,,Bug,Bug,241,stunspore,spore,megadrain,bodyslam,210,...,blizzard,thunderwave,bodyslam,184,189,207,207,169,6,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
1,graveler,,Rock,Rock,252,earthquake,bodyslam,explosion,rockslide,231,...,blizzard,thunderwave,bodyslam,184,189,207,207,169,6,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
2,paras,,Bug,Bug,145,stunspore,spore,megadrain,bodyslam,210,...,psychic,sleeppowder,stunspore,138,199,169,169,138,4,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
3,mewtwo,,Psychic,Psychic,72,psychic,amnesia,thunderbolt,blizzard,199,...,psychic,sleeppowder,stunspore,138,199,169,169,138,4,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
4,chansey,,Normal,Normal,98,sing,softboiled,thunderwave,icebeam,75,...,blizzard,thunderwave,bodyslam,184,189,207,207,169,2,"[Gen 1] Random Battle-10071,51491,20071,43286-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11095,charizard,,Fire,Fire,101,hyperbeam,swordsdance,fireblast,bodyslam,205,...,blizzard,surf,seismictoss,171,201,174,174,40,1,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11096,moltres,,Fire,Fire,297,reflect,fireblast,hyperbeam,agility,230,...,blizzard,surf,seismictoss,171,201,174,174,40,1,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11097,moltres,,Fire,Fire,297,reflect,fireblast,hyperbeam,agility,230,...,blizzard,surf,seismictoss,171,201,174,174,40,0,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
11098,charizard,,Fire,Fire,278,hyperbeam,swordsdance,fireblast,bodyslam,205,...,swordsdance,agility,bodyslam,176,161,165,165,169,6,"[Gen 1] Random Battle-9939,58,62081,43766-turn..."
